<a href="https://colab.research.google.com/github/aldojda/Processing-DataClassification/blob/main/01_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
map_uanl = {'fcfm': 'fcfm',
 'facpya': 'facpya',
 'farq': 'farq',
 'facultad de medicina': 'facmed',
 'filosofia y letras': 'ffyl',
 'facultad de ingenieria civil': 'fic',
 'facultad de ciencias de la comunicacin': 'fcc',
 'faculta de ciencias biologicas': 'fcb',
 'facultad de psicologia': 'fapsi',
 'facultad de ciencias qumicas': 'fcq',
 'facultad de derecho y criminologa': 'facdyc',
 'facultad de odontologia': 'fo',
 'faspyn': 'faspyn',
 'facultad de ciencias politicas y admon pub': 'facpya',
 'facultad de msica': 'famus',
 '': '',
 'facultad de medicina veterinaria y zootecnia': 'vet',
 'facultad de ciencias fisicomatematicas': 'fcfm',
 'facultad de ciencias de la tierra': 'fct',
 'facultad de ingeniera mecanica yelectrica': 'fime',
 'facdyc': 'facdyc',
 'facpyap': 'facpya',
 'facultad de ciencias quimicas': 'fcq',
 'facultad de agronoma': 'agro',
 'fct': 'fct',
 'artes visuales': 'fav',
 'facultad de arquitectura y diseo industrial': 'farq'}


In [36]:
map_mes = {
    'Ene': '01',
    'Feb': '02',
    'Mar': '03',
    'Abr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Ago': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dic': '12'
}

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
escuelas = pd.read_csv("drive/MyDrive/01 - Maestría Ciencia de Datos/proyectoSentimentAnalysisUANL/02_escuelasURL.csv", encoding = "Latin1")
escuelas = escuelas[~ escuelas.Escuela.str.lower().str.contains("prep")]
escuelas.Escuela =  escuelas.Escuela.str.lower().str.replace("uanl", "").str.strip()
escuelas.Escuela = escuelas.Escuela.str.replace(r'[^a-zA-Z0-9\s]', '', regex= True).str.strip()
escuelas.Escuela =  escuelas.Escuela.map(map_uanl).str.upper()
escuelas.to_json('escuelas.json', orient='records', lines=True)

In [39]:
# traemos mapeo de materias clean
materias = pd.read_csv("drive/MyDrive/01 - Maestría Ciencia de Datos/proyectoSentimentAnalysisUANL/03_materias.csv", encoding = "Latin1")
materias.head(2)
# traemos información de maestros
prof = pd.read_csv("drive/MyDrive/01 - Maestría Ciencia de Datos/proyectoSentimentAnalysisUANL/01_maestrosURL.csv")
prof.head(2)
# leemos los comentarios por maestro
comments = pd.read_csv("/content/drive/MyDrive/01 - Maestría Ciencia de Datos/proyectoSentimentAnalysisUANL/00_comentarios_misprofesores.csv")
comments["month"] = comments.Date.apply(lambda x: x.split("/")[1]).map(map_mes)
comments["day"] = comments.Date.apply(lambda x: x.split("/")[0])
comments["year"] = comments.Date.apply(lambda x: x.split("/")[2])
comments.Date = pd.to_datetime(comments.year + "-" + comments.month + "-" + comments.day, format = "%Y-%m-%d")
comments.drop(columns = ["month", "day", "year", "Unnamed: 0"], inplace = True)
print(comments.shape)
# eliminamos datos nulos
comments =  comments[comments.Comments.notna()].copy()
# convertimos a lowecase
comments["Comments"] = comments["Comments"].str.lower()
# reemplazamos datos no numéricos
comments["Comments"] = comments["Comments"].str.replace(r'[^a-zA-Z0-9\s]', '', regex= True).str.strip()
print(comments.shape)
# add scool Information
comments = comments.merge(prof[["i", "escuela"]], how = "left", left_on = "id", right_on = "i", validate = "m:1")
# removemos datos de preparatoria
comments = comments[~ comments.escuela.str.lower().str.contains("prep")]
comments.escuela =  comments.escuela.str.lower().str.replace("uanl", "").str.strip()
comments["escuela"] = comments["escuela"].str.replace(r'[^a-zA-Z0-9\s]', '', regex= True).str.strip()
comments.escuela =  comments.escuela.map(map_uanl).str.upper()
comments = comments[comments.escuela != ""].copy()
comments["Class Name"] =  comments["Class Name"].str.lower().str.replace(r'[^a-zA-Z\s]', '', regex= True).str.strip()
comments =  comments[comments["Class Name"] != ""].copy()
comments = comments.sort_values(by = "Class Name")
comments["map_materias"] = comments["Class Name"].map(materias.set_index("materia")["map"].to_dict())
print(comments.shape)
#comments.date = pd.to_datetime(comments.date)
comments.head(2)

(27223, 11)
(19626, 11)
(17796, 14)


,Date,General Quality,Ease,Class Name,Attendance,Received Grade,Interest,Comments,Helpful Count,Not Helpful Count,id,i,escuela,map_materias
4073,2015-07-22,8.0,10.0,aa bocetos color,Obligatoria,NaN,Alto,es algo raro pero pasas porque pasas,0,0,55310,55310,FARQ,datos
2991,2017-11-11,10.0,10.0,aartes,No obligatoria,NaN,Alto,lo que anota en el pizarron es lo que viene en...,0,0,92815,92815,FARQ,artes


In [31]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis',
                      model="nlptown/bert-base-multilingual-uncased-sentiment")
#sentences = comments.Comments.to_list()
#results = classifier(sentences)

In [42]:
x = pd.DataFrame(results)
x["Comments"] = sentences
x = x.drop_duplicates(subset = "Comments", keep = "first")

In [43]:
comments = comments.merge(x, how = "left", on = "Comments", validate = "m:1")
comments["label"] = comments["label"].str[0].astype(int)
comments["sentimiento"] = np.where(comments.label<= 3, "negativo",
                             np.where(comments.label == 4, "neutro", "postivo"))
comments.head()

,Date,General Quality,Ease,Class Name,Attendance,Received Grade,Interest,Comments,Helpful Count,Not Helpful Count,id,i,escuela,map_materias,label,score,sentimiento
0,2015-07-22,8.0,10.0,aa bocetos color,Obligatoria,NaN,Alto,es algo raro pero pasas porque pasas,0,0,55310,55310,FARQ,datos,3,0.524987,negativo
1,2017-11-11,10.0,10.0,aartes,No obligatoria,NaN,Alto,lo que anota en el pizarron es lo que viene en...,0,0,92815,92815,FARQ,artes,5,0.260514,postivo
2,2023-12-18,10.0,10.0,aavd,No obligatoria,9.5,Alto,sin ninguna duda el mejor profesor que he teni...,0,0,110702,110702,FCFM,artes,5,0.826551,postivo
3,2021-08-30,9.0,8.0,aavd y mad,No obligatoria,23.0,Alto,bueno llevo 5 semanas de estar con l en dos ma...,0,0,129286,129286,FCFM,artes,4,0.447667,neutro
4,2020-12-10,7.0,8.0,abastecimientos,No obligatoria,9.0,Medio,aheli es super buena onda sabe mucho de su rea...,0,0,120161,120161,FACPYA,abastos,4,0.370662,neutro


In [44]:
comments.to_json('datos.json', orient='records', lines=True)